In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers


'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [9]:
#!pip install torch==1.7.0
!pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [6]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import KFold
import wget
import numpy as np

In [48]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
wget.download('https://raw.githubusercontent.com/donghyun-daniel/4_party_project/master/Review_crawler/csv_output/Preprocessing_review_pos_neg.csv')

100% [..........................................................................] 9183137 / 9183137

'Preprocessing_review_pos_neg.csv'

In [7]:
device = torch.device("cuda")

In [106]:
class NSMCDataset(Dataset):
  
    def __init__(self, csv_file):
        # 일부 값중에 NaN이 있음...
        self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
        # 중복제거
        self.dataset.drop_duplicates(subset=['document'], inplace=True)
        self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:3].values
        text = row[0]
        y = row[1]

        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, y


    
def data_concat(naverTrainData, naverTestData, hotelData): 
    #data_concat("ratings_train.txt", "ratings_test.txt", "Preprocessing_review_pos_neg.csv")
    trainRate = 0.5
    
    data1 = pd.read_table(naverTrainData)
    data2 = pd.read_table(naverTestData)
    data3 = pd.read_csv(hotelData)
    
    data3 = data3.drop(['len_text'], axis=1)
    data3['id'] = 1
    data3 = data3[['id', 'Text', 'Label']]
    data3.columns = ['id','document','label']
    data3 = data3.iloc[np.random.permutation(data3.index)].reset_index(drop=True)
    
    data3Train = data3.sample(frac = trainRate, random_state=2000)
    data3Test = data3.drop(data3Train.index)
    
    trainData = pd.concat([data1, data3Train], ignore_index=True)
    testData = pd.concat([data2, data3Test], ignore_index=True)
    
    trainData.to_csv('sampleTrain.txt', sep = '\t', index = False)
    testData.to_csv('sampleTest.txt', sep = '\t', index = False)

In [107]:
data_concat("ratings_train.txt", "ratings_test.txt", "Preprocessing_review_pos_neg.csv")

In [108]:
train_dataset = NSMCDataset("sampleTrain.txt")
test_dataset = NSMCDataset("sampleTest.txt")

                 id          label
count  1.964910e+05  196491.000000
mean   5.043462e+06       0.500725
std    3.885104e+06       0.500001
min    1.000000e+00       0.000000
25%    1.000000e+00       0.000000
50%    5.835929e+06       1.000000
75%    8.768903e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  9.955000e+04  99550.000000
mean   3.334551e+06      0.503978
std    3.957116e+06      0.499987
min    1.000000e+00      0.000000
25%    1.000000e+00      0.000000
50%    1.000000e+00      1.000000
75%    7.524211e+06      1.000000
max    1.027809e+07      1.000000


In [109]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [110]:
# 에러나도 그냥 넘어가면 됨
model.load_state_dict(torch.load("model.pt"))



FileNotFoundError: [Errno 2] No such file or directory: 'model.pt'

In [111]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [113]:
# GPU 메모리 터지는거 방지
gc.collect()
torch.cuda.empty_cache()

In [ ]:
epochs = 3
batch_size = 64

optimizer = AdamW(model.parameters(), lr=1e-5)

# 10 fold cross val
kfold = KFold(n_splits=10, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

kfold_losses = []
kfold_acc = []


for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
    print('----------------------------------')
    print(fold)

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_subsampler)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_subsampler)
    
    losses = []
    accuracies = []
    
    for i in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0
        batches = 0

        model.train()

        for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
            loss = F.cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)

            batches += 1
            if batches % 100 == 0:
                print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

        losses.append(total_loss)
        accuracies.append(correct.float() / total)
        print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)
    loss_avg = sum(losses) / len(losses)
    acc_avg = sum(accuracies) / len(accuracies)
    kfold_losses.append(loss_avg)
    kfold_acc.append(acc_avg)
    
    print('----------------------------------')

----------------------------------
0


  0%|          | 0/2764 [00:00<?, ?it/s]

E:\Build\Project\programmers\4_party_project\env\lib\site-packages\transformers\tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 26.482181921601295 Accuracy: tensor(0.8919, device='cuda:0')
Batch Loss: 52.13588486611843 Accuracy: tensor(0.8937, device='cuda:0')
Batch Loss: 79.60805383324623 Accuracy: tensor(0.8919, device='cuda:0')


In [117]:
kfold_losses, kfold_acc

([1178.8993189930916, 934.6577288433909, 857.1547686308622],
 [tensor(0.8340, device='cuda:0'),
  tensor(0.8714, device='cuda:0'),
  tensor(0.8839, device='cuda:0')])

In [118]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/6222 [00:00<?, ?it/s]

Accuracy: tensor(0.8987, device='cuda:0')


In [ ]:
torch.save(model.state_dict(), "model.pt")

## 여기까지

In [30]:
y_pred

tensor([[ 0.4687, -0.4607],
        [ 2.0620, -2.2696],
        [-0.1525,  0.2123],
        [ 1.0173, -1.0771],
        [ 2.4705, -2.8007]], device='cuda:0', grad_fn=<AddmmBackward>)

In [33]:
_, pre=torch.max(y_pred, 1)

In [34]:
pre

tensor([0, 0, 1, 0, 0], device='cuda:0')

In [35]:
train_dataset

In [36]:
train_loader

In [39]:
input_ids_batch

tensor([[    2,  3711, 29950,  ...,     0,     0,     0],
        [    2,  2207, 30006,  ...,     0,     0,     0],
        [    2,    63,  3008,  ...,     0,     0,     0],
        [    2,  9387, 29950,  ...,     0,     0,     0],
        [    2, 12466, 29993,  ...,     0,     0,     0]])

In [40]:
attention_masks_batch

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [41]:
y_batch

tensor([0, 0, 1, 1, 0], device='cuda:0')

In [42]:
for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    break

  0%|          | 0/4569 [00:00<?, ?it/s]

In [45]:
input_ids_batch

tensor([[    2,   494, 29950,  ...,     0,     0,     0],
        [    2,   800, 29952,  ...,     0,     0,     0],
        [    2,   403, 29992,  ...,     0,     0,     0],
        ...,
        [    2,  5980,  1975,  ...,     0,     0,     0],
        [    2,  5813, 30136,  ...,     0,     0,     0],
        [    2,   620, 30984,  ...,     0,     0,     0]])

In [46]:
attention_masks_batch

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [47]:
y_batch

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 0])

# 올릴 방법

- 어근 치환 ***
- k-fold crossval 

- pretrain 모델 변경 ***

- fine tuning
// - 데이터 추가 영화, 호텔, 쇼핑 데이터




In [145]:
data = [1,2,3,4,5,6,7,8,9,10]

[1 2 3 4 5 6 7 8 9] [0]
9
7
2
5
1
3
4
8
6
[0 1 2 4 5 6 7 8 9] [3]
5
0
6
7
2
1
4
9
8
[0 1 2 3 4 5 6 7 9] [8]
1
6
4
3
7
0
9
5
2
[0 1 2 3 4 5 6 7 8] [9]
5
1
4
2
8
0
3
6
7
[0 1 2 3 4 6 7 8 9] [5]
7
1
0
2
8
6
9
4
3
[0 1 2 3 4 5 6 8 9] [7]
9
1
4
6
3
8
0
2
5
[0 1 3 4 5 6 7 8 9] [2]
5
8
0
1
7
4
6
9
3
[0 1 2 3 4 5 7 8 9] [6]
8
7
2
3
5
4
0
1
9
[0 1 2 3 5 6 7 8 9] [4]
1
8
2
3
0
5
7
9
6
[0 2 3 4 5 6 7 8 9] [1]
8
0
2
6
3
9
7
4
5
